Data location: https://kingcounty.gov/en/dept/metro/rider-tools/mobile-and-web-apps#toc-developer-resources

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mpldates
import seaborn as sns
import plotly as py
from numpy.lib.recfunctions import drop_fields

In [ ]:
#solve the 'fuzzy text' issue by increasing the DPI with a whole format change (from Matplotlib)
%config InlineBackend.figure_format='retina'

In [ ]:
# Import all CSVs
GTFS_path_location = "King_County_GTFS_feed/"

agency = pd.read_csv(GTFS_path_location + "agency.txt")
calendar = pd.read_csv(GTFS_path_location + "calendar.txt")
calendar_dates = pd.read_csv(GTFS_path_location + "calendar_dates.txt")
fare_attributes = pd.read_csv(GTFS_path_location + "fare_attributes.txt")
fare_rules = pd.read_csv(GTFS_path_location + "fare_rules.txt")
routes = pd.read_csv(GTFS_path_location + "routes.txt")
shapes = pd.read_csv(GTFS_path_location + "shapes.txt")
stop_times = pd.read_csv(GTFS_path_location + "stop_times.txt")
stops = pd.read_csv(GTFS_path_location + "stops.txt")
trips = pd.read_csv(GTFS_path_location + "trips.txt")


### Treatment
#### convert numbers to datetime

In [ ]:
calendar["start_date"] = pd.to_datetime(calendar["start_date"], format = "%Y%m%d")
calendar["end_date"] = pd.to_datetime(calendar["end_date"], format = "%Y%m%d")
calendar_dates["date"] = pd.to_datetime(calendar_dates["date"], format = "%Y%m%d")


### Combine both Calendar dataframes

In [ ]:
# Map column values from integer to date-time
# build dataframe calendar_full
#    index = all service_id (outer join of both service_id sets)
#    column = all dates in range (min start date, max end date)
#    True/False or 1/0 if service is given
# fill in calendar values
# fill in calendar_date values

# collapse down to weekdays (percent of day X has service)

# heatmap of calendar_full
# heatmap of calendar_week
# set dataframe calendar_week as booleans

In [ ]:
def create_calendar_full (calendar, calendar_dates):
    """
    creates a dataframe with:
    Columns:
        [0] service_id
        [1:-1] date-time objects, each date from earliest date to latest date in Calendar
    Index:
        integers from 0
    Values:
        - service ID is filled with all unique values from calendar and calendar_dates
        - all other values are NaN
    
    """
    # Create variable calendar_full_columns
    # Is a series with every date (in date time) from the first date available in calendar to the last
    # Calendar should have the full range of dates covered by the database
    cal_range_max = calendar.loc[:,["start_date","end_date"]].max().max()
    cal_range_min = calendar.loc[:,["start_date","end_date"]].min().min()
    cal_range = pd.date_range(start = cal_range_min, end = cal_range_max, inclusive="both")
    calendar_full_columns = pd.Series(["service_id"] + list(cal_range))

    # Create variable service_id_list
    # Is a series with every service id in calendar and calendar_dates
    # filtered for only unique 
    service_id_list = pd.merge(calendar.loc[:,["service_id"]], calendar_dates.loc[:,["service_id"]], how = "outer")
    service_id_list.drop_duplicates(inplace = True)
    service_id_list.reset_index(inplace = True, drop = True)
    
    # Create dataframe
    calendar_full = pd.DataFrame(data = service_id_list, columns = calendar_full_columns)
    
    return calendar_full


def add_calendar (calendar_full, calendar):
    """
    assumes calendar_full's index is intergers indexed by 1
    assumes calendar_full's columns are as such:
      [0] service_id
      [1:-1] date-time objects
    
    """
    # iterate through entire calendar_full, sans service_id column
    # where i is index number, and j is column number
    for i in range(0,calendar_full.index.max() + 1):
        for j in range(1,len(calendar_full.columns)):
            # Target in Calendar, based on numerical index/column of calendar_full
            target_cal_index = calendar["service_id"] == calendar_full["service_id"][i]
            target_cal_column = calendar_full.columns[j].day_name().lower()
            calendar_full.iloc[i,j] = calendar.loc[target_cal_index, target_cal_column]
   
    return calendar_full

def add_calendar_dates (calendar_full, calendar_dates):
    """
    assumes calendar_full's index is intergers indexed by 1
    assumes calendar_full's columns are as such:
      [0] service_id
      [1:-1] date-time objects
    
    assumes calendar_dates only has unique combinations or service_id + date.
    this function will ignore non-unique combinations
    
    """
    # iterate through entire calendar_full, sans service_id column
    # where i is index number, and j is column number
    for i in range(0,calendar_full.index.max() + 1):
        for j in range(1,len(calendar_full.columns)):
            # find index in calendar_dates that matches a particular cell in calendar_full
            # target index is a series of booleans
            target_service_id = calendar_dates["service_id"] == calendar_full.iloc[i,0]
            target_date = calendar_dates["date"] == calendar_full.columns[j]
            target_index = target_service_id & target_date
            # check to make sure there is a target (target_index has one True value)
            if target_index.sum() == 1:
                # exception is value of exception type (as per data dictionary, is only 1 or 2)
                exception = calendar_dates.loc[target_index, "exception_type"].iloc[0]
                if exception == 1:
                    # service added
                    calendar_full.iloc[i,j] = 1        
                elif exception == 2:
                    # service removed
                    calendar_full.iloc[i,j] = 0
    return calendar_full


def cal_full_alter_axis(x):
    """
    Pull service id out of the columns and use it as the index
    """
    x = x.set_index("service_id").copy()
    return x

def create_calendar_week(calendar_full,binary = False):
    """
    create DF calendar_week.
    It's in the same format as "calendar"
    values are averages
    
    If binary is True a field of binary values are returned.  
    Where the values are the averages round to 0 or 1
    """
    calendar_week = pd.DataFrame(data = calendar_full["service_id"], 
                                 columns = ["service_id"]
                                )

    weekday_list = ["monday","tuesday","wednesday","thursday","friday","saturday","sunday"]

    for day in weekday_list:
        # And then add the two edge dates
        filter_values = [True] + [True if x == day else False for x in calendar_full.columns[1:]]
        # create DF of serviceid plus only days of specified week
        filtered_cal = calendar_full.loc[:,filter_values]
        #average day of week
        add_day = filtered_cal.iloc[:,1:].T.groupby(lambda x: True).mean().T
        if binary == True:
            add_day = add_day.map(lambda x: round(x))
        add_day.columns = [day]
        calendar_week[day] = add_day

    calendar_week["start_date"] = [calendar_full.columns[1]] * calendar_week.shape[0]
    calendar_week["end_date"] = [calendar_full.columns[-1]] * calendar_week.shape[0]

    return calendar_week

In [ ]:
# create calendar_full and populate it
calendar_full = create_calendar_full(calendar, calendar_dates)
calendar_full = calendar_full.pipe(add_calendar, calendar = calendar) \
    .pipe(add_calendar_dates, calendar_dates = calendar_dates)
# create an alternate version for graphing and analysis
cal_full_alt = cal_full_alter_axis(calendar_full)
# Create calendar week
calendar_week = create_calendar_week(calendar_full, binary = True)
calendar_week_mean = create_calendar_week(calendar_full, binary = False)


In [ ]:
sns.set_context('paper')
sns.color_palette('blend:#3e6e64,#c3d4d0', as_cmap = True)
ax = (sns.heatmap(calendar_week_mean.set_index("service_id").iloc[:,:-2], 
            yticklabels = True, 
            cbar = False, 
            cmap = sns.color_palette('blend:#cfd8d6,#3e6e64', as_cmap = True), 
            linewidth = 0.5
           )
    )
ax.set(xlabel = "Weekday",
      ylabel = "Service ID",
      title = "Heatmap: Likelyhood of Service"
     )
plt.show()

In [ ]:
# Order service ids by how many trips they are assigned to, in descending order.
# service ids with no trips are appended to the end
x = pd.Index(set(cal_full_alt.index).difference(set(trips.index)))
x = trips.value_counts("service_id").index.append(x)
cal_full_alt_graph = cal_full_alt.reindex(x, copy = True)

# change timestamps in column axis to strings for visualization
cal_full_alt_graph.columns = [y.strftime("%b-%d") for y in cal_full_alt.columns]



sns.set_context('paper')
sns.color_palette('blend:#3e6e64,#c3d4d0', as_cmap = True)
ax = (sns.heatmap(cal_full_alt_graph.iloc[:,0:31], 
            yticklabels = True, 
            cbar = False, 
            cmap = sns.color_palette('blend:#cfd8d6,#3e6e64', as_cmap = True), 
            linewidth = 0.5
           )
    )
ax.set(xlabel = "Date",
      ylabel = "Service ID",
      title = "Service heatmap"
     )

#ax.xaxis.set_major_formatter(mpldates.DateFormatter("%d-%b")) doesnt work yet, moving on
xtix = ax.get_xticks()
ax.set_xticks(xtix[::7])
ax.set_xticklabels(cal_full_alt_graph.iloc[:,0:31].columns[::7])
ax.tick_params(axis = 'x', rotation = 0)
plt.show()

In [ ]:
# I have a map of the average service_ids for any given weekday (or specific day)
# For any given route and day, find all the trips.
# next lets get the start time for all of them

In [ ]:

#attach routes to trips on route_id for route_names

#combine stop_times to trips on trip_id tring to pull the first arrival_time


In [ ]:
def trips_merged(trips, stop_times, routes, calendar_week):
    """
    
    """
    #pull stop times
    stop_times["index_og"] = stop_times.index
    stop_times_index = stop_times.groupby(["trip_id"])[["arrival_time","index_og"]].min("arrival_time")["index_og"]
    trip_times = stop_times.loc[index1,["trip_id","arrival_time"]]

    #pull route names
    trip_routes = routes.loc[:,["route_id","route_short_name","route_desc"]]
    
    #pull service_weekdays
    # just drop it in
    
    trips_alt = pd.merge(left = trips, right = trip_times, how = "inner", on = "trip_id")
    
    trips_alt = pd.merge(left = trips_alt, right = trip_routes, how = "inner", on = "route_id")
    
    trips_alt = pd.merge(left = trips_alt, right = calendar_week, how = "inner", on = "service_id")

    return trips_alt
    
def routes_on_a_day(trips, weekday, route,direction):
    return trips.loc[((trips["route_short_name"] == route) & 
                      (trips[weekday] == 1) & \
                      (trips["direction_id"] == direction)),
                     :].sort_values("arrival_time")
    
def stop_time_to_datetime(value):
    time = value
    hour = int(time[0:2])
    if hour > 23:
        day = "1970-01-02"
        hour = hour - 24
        time = str(hour) + time[2:-1]
    else:
        day = "1970-01-01"
    return pd.to_datetime((day + " " + time))

def create_time_delta(df, time_col, delta_col):
    """
    outputs the same df, with a new column, delt_col
    
    the timedelta for each point is the time elapsed BEFORE the point.
    delta = Tn - T(n-1)
    
    df: dataframe with the datetime to build from
        dataframe the delta time will be added to
        
    time_col: name of the column with datetime information.
              must be in timestamp format
              
    delta_col: the name of the new column
    """
    # Create a column of the differences
    df[delta_col] = df[time_col].diff()
    
    # create a timedelta object equal to one day 
    one_day = pd.to_timedelta(1, unit='d')
    
    # calculate the time differential between the last and the first value.
    df.loc[df.index[0],delta_col] = \
                                    df[time_col].iloc[0] - \
                                    (df[time_col].iloc[-1] - one_day)
    
    return df

In [ ]:
route_choice = "D Line"
weekday_choice = "monday"
direction_choice = 1

# merge data desired
trips_alt = trips_merged(trips, stop_times, routes, calendar_week)
# filter to the desired information
route_day = routes_on_a_day(trips_alt, weekday_choice, route_choice, direction_choice)
# filter to desired columns
route_day = route_day.loc[:,["service_id","arrival_time","route_short_name"]]
# Convert 'arrival time' to datetime
route_day['arrival_time'] = route_day['arrival_time'].map(stop_time_to_datetime)
# create time_delta: difference between arrival times
route_day = create_time_delta(route_day, "arrival_time", "time_delta")
route_day.head()

In [ ]:
# Convert timestamp and deltatime objects to numbers.  Graphing cannot understand datetime objects.
route_day["delta_val"] = route_day["time_delta"].map(lambda x: x.seconds/60)
route_day["time_val"] = route_day["arrival_time"].map(lambda x: x.hour + x.minute/60)

sns.set_style("darkgrid")
ax =(sns.lineplot(data = route_day,
                  x = 'time_val',
                  y = 'delta_val'
                 )
    )

ax.set(xlabel = "Hour of the Day",
      ylabel = "minutes",
      title = "time between buses - D Line - Weekday - dir 1"
     )

#xtix = ax.get_xticks()
ax.set_yticks([0,5,10,15,30,45,60])
ax.set_ylim(0,65)
ax.set_xticks([0,5,7,9,12,16,18,20,21,22,24])
ax.set_xlim(0,24)
#ax.set_xticklabels(cal_full_alt_graph.iloc[:,0:31].columns[::7])
#ax.tick_params(axis = 'x', rotation = 0)

plt.show()

In [ ]:
# Make the graph prettier
# try to make it really compact (so it can go beside a map)
# smooth it out into something usable for riders


# things to look further into: does the number of stops change for a route?


In [ ]:
# Graph multiple routes on the same axis
